In [203]:
import pandas as pd
from sklearn.model_selection import train_test_split
orig_data = pd.read_pickle("./unprocessed.bz2")
question_headers = []
non_question_headers = []
for h in list(orig_data):
    if h[0] == 'q':
        question_headers.append(h)
    elif h[0] != 'p':
        non_question_headers.append(h)
print '\',\''.join(non_question_headers)

# non_question_headers = ['d_smokes','d_education_type','d_drinks','d_gender','d_religion_type','d_drugs']

d_astrology_seriosity','d_education_phase','d_gender','d_income','d_religion_type','d_drugs','lf_want','d_smokes','d_astrology_sign','d_country','d_drinks','d_age','d_ethnicity','d_offspring_current','d_bodytype','lf_max_age','lf_for','d_job','d_orientation','d_languages','d_relationship','d_offspring_desires','lf_location','lf_min_age','d_education_type','lf_single','d_religion_seriosity','CA','gender_orientation','gender','race','gender2','gender2_num','CA_items


In [204]:
from sklearn.model_selection import train_test_split
import numpy as np

question = 'q9688'
question_data = orig_data[[question] + non_question_headers]
#drop rows with Nan for question
question_data = question_data.dropna(subset=[question])
question_data = question_data.replace(np.nan, 'Nan', regex=True)
print question_data.shape


#convert from category to int
obj_columns = question_data.select_dtypes(['object']).columns
question_data[obj_columns] = question_data[obj_columns].astype('category')
cat_columns = question_data.select_dtypes(['category']).columns #

#https://stackoverflow.com/questions/51102205/how-to-know-the-labels-assigned-by-astypecategory-cat-codes
question_categories = dict(enumerate(question_data['q9688'].cat.categories))
print question_categories

question_data_codes = question_data.copy()
question_data_codes[cat_columns] = question_data_codes[cat_columns].apply(lambda x: x.cat.codes)


question_data.iloc[:5]

(55697, 7)
{0: 'No', 1: 'Yes', 2: 'Yes, but only soft stuff like marijuana'}


,q9688,d_smokes,d_education_type,d_drinks,d_gender,d_religion_type,d_drugs
0,No,No,masters program,Socially,Woman,-,Never
2,Yes,Yes,Nan,Rarely,Woman,Other,Nan
4,Yes,No,Ph.D program,Socially,Woman,Atheism,Sometimes
5,Yes,No,university,Often,Man,Agnosticism,Never
8,No,Nan,Nan,Nan,Woman,-,Never


In [208]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x = question_data_codes[non_question_headers]
y = question_data_codes[question]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
clf = tree.DecisionTreeClassifier(max_depth=6)
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

labels=list(question_data[question].cat.categories)
print labels

cm = confusion_matrix(y_test, y_pred)

print np.around(cm.astype(float)/np.sum(cm),decimals=2)
print 'acc',accuracy_score(y_test, y_pred)
print 'acc random preds', accuracy_score(np.random.permutation(list(y_test)), np.random.permutation(list(y_pred)))



['No', 'Yes', 'Yes, but only soft stuff like marijuana']
[[0.27 0.   0.13]
 [0.02 0.03 0.07]
 [0.17 0.03 0.28]]
acc 0.5842010771992818
acc random preds 0.42405745062836625


In [212]:
tree.export_graphviz(clf, out_file='tree.dot')
print clf.feature_importances_

[0.06364804 0.00612783 0.03662231 0.06351161 0.13274847 0.69734173]
